## writer api's.

#### Mode of writitng back to disk -overwrite, append,ignore,errorIfExists

#### patitioning, bucketing, fileformat

1)we use partition By based on a column while writing the file back to the disk.

2)multiple folders will e created based on the unique values in that col

3)Each folder will multiple file based on the number of partitions that the df had when it was created

4)When there are a large number of distinct values, we can't use partition BY.

5)Thats why we have bucketing

#### 1)Open a Spark Notebook and create a spark base dataframe by reading all the files under /public/sms/users folder. 

In [1]:
! hadoop fs -ls -h /public/sms/users

Found 10 items
-rw-r--r--   2 hdfs supergroup     28.1 M 2021-01-28 11:30 /public/sms/users/users_01.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 09:18 /public/sms/users/users_02.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 09:07 /public/sms/users/users_03.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 09:00 /public/sms/users/users_04.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 11:13 /public/sms/users/users_05.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 11:03 /public/sms/users/users_06.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 11:01 /public/sms/users/users_07.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 08:51 /public/sms/users/users_08.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 11:19 /public/sms/users/users_09.json
-rw-r--r--   2 hdfs supergroup     28.2 M 2021-01-28 09:20 /public/sms/users/users_10.json


In [29]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark= SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [3]:
spark

In [30]:
from pyspark.sql.types import StructType, StructField, StringType,IntegerType,ArrayType

In [31]:
 #creating schema for reading the json file
IntegerType, ArrayType
 users_schema = StructType([
 StructField("user_id", IntegerType(), nullable=False),
 StructField("user_first_name", StringType(), nullable=False),
 StructField("user_last_name", StringType(), nullable=False),
 StructField("user_email", StringType(), nullable=False),
 StructField("user_gender", StringType(), nullable=False),
 StructField("user_phone_numbers", ArrayType(StringType()),
 nullable=True),
 StructField("user_address", StructType([
 StructField("street", StringType(), nullable=False),
 StructField("city", StringType(), nullable=False),
 StructField("state", StringType(), nullable=False),
 StructField("postal_code", StringType(), nullable=False),
 ]), nullable=False)
 ])

In [32]:
base_df = spark.read \
.format('json') \
.options(schema=users_schema) \
.load("/public/sms/users/*")

In [7]:
# 1)Do check how many partitions are created  in your dataframe.
base_df.rdd.getNumPartitions()

4

In [8]:
# 2) Do some basic analysis and find out the following

In [9]:
base_df.show(truncate=False)

+----------------------------------------------------------+--------------------------------+---------------+-----------+-------+--------------+------------------------------------------------------------+
|user_address                                              |user_email                      |user_first_name|user_gender|user_id|user_last_name|user_phone_numbers                                          |
+----------------------------------------------------------+--------------------------------+---------------+-----------+-------+--------------+------------------------------------------------------------+
|{Dallas, 75358, Texas, 8 Warrior Drive}                   |ecutsforth0@wisc.edu            |Eirena         |Female     |200001 |Cutsforth     |[4197404036, 9173828061, 4027467939, 3371963735]            |
|{Joliet, 60435, Illinois, 66 Prairieview Terrace}         |mshopcott1@hexun.com            |Marja          |Female     |200002 |Shopcott      |[9542037028, 2128289053, 4068088

In [33]:
from pyspark.sql.functions import col,size
base_df.withColumn("user_street",col("user_Address.street")) \
.withColumn("user_city",col("user_address.city")) \
.withColumn("user_state", col("user_Address.state")) \
.withColumn("user_postal_code", col("user_address.postal_code")) \
.withColumn("num_phn_numbers",
size(col("user_phone_numbers"))).createOrReplaceTempView("users_vw")

In [11]:
spark.sql("select * from users_vw")

user_address,user_email,user_first_name,user_gender,user_id,user_last_name,user_phone_numbers,user_street,user_city,user_state,user_postal_code,num_phn_numbers
"{Dallas, 75358, T...",ecutsforth0@wisc.edu,Eirena,Female,200001,Cutsforth,"[4197404036, 9173...",8 Warrior Drive,Dallas,Texas,75358,4
"{Joliet, 60435, I...",mshopcott1@hexun.com,Marja,Female,200002,Shopcott,"[9542037028, 2128...",66 Prairieview Te...,Joliet,Illinois,60435,5
"{Shawnee Mission,...",dtointon2@ucsd.edu,Dawn,Female,200003,Tointon,"[9523035647, 2134...",18 Ronald Regan Hill,Shawnee Mission,Kansas,66225,3
"{Saint Paul, 5516...",gleaman3@360.cn,Goldi,Female,200004,Leaman,"[2027069459, 7042...",7696 Calypso Junc...,Saint Paul,Minnesota,55166,5
"{Albuquerque, 871...",bhallagan4@livejo...,Brewster,Male,200005,Hallagan,"[8134746319, 2152...",942 Emmet Park,Albuquerque,New Mexico,87110,2
"{Virginia Beach, ...",fglashby5@deviant...,Florence,Female,200006,Glashby,"[7571763565, 7134...",664 Ridge Oak Circle,Virginia Beach,Virginia,23454,5
"{Young America, 5...",zphilimore6@webed...,Zollie,Male,200007,Philimore,"[7865450177, 8019...",4 Continental Drive,Young America,Minnesota,55557,2
"{Tucson, 85725, A...",framelot7@alexa.com,Ferdinand,Male,200008,Ramelot,[8164055187],44698 Arapahoe Park,Tucson,Arizona,85725,1
"{null, null, null...",rgreystoke8@digg.com,Ruddy,Male,200009,Greystoke,null,null,null,null,null,-1
"{North Las Vegas,...",gbeeho9@webmd.com,Gilles,Male,200010,Beeho,"[5599886694, 5201...",9430 Reinke Park,North Las Vegas,Nevada,89087,2


In [12]:
# a. total number of records in the dataframe
spark.sql("select count(*) from users_vw")

count(1)
1000000


In [34]:
# b. how many users are from the state New York
spark.sql("select count(distinct user_id) as unique_id from users_vw where user_state = 'New York' ")

unique_id
49576


In [16]:
#  c. which state has maximum number of postal codes
spark.sql("""select user_state,count(distinct user_postal_Code) as postal_cnt
 from users_vw
 group by user_state order by postal_cnt desc limit 1""")

user_state,postal_cnt
California,206


In [17]:
#  d. which city has the most number of users

spark.sql(''' select user_city, count(distinct user_id) as uniq_id  from users_vw
where user_city is not null 
group by user_city 
order by uniq_id desc limit 1
''')

user_city,uniq_id
Washington,28504


In [35]:
spark.sql(''' select user_state, count(distinct user_postal_code) as uniq_psc  from users_vw
group by user_state order by uniq_psc desc limit 1
''')

user_state,uniq_psc
California,206


In [19]:
# e. how many users have email domain as bizjournals.com

In [36]:
spark.sql(''' select count(*) as  from users_vw
where user_email like '%bizjournals.com'  
''')

as
2015


In [37]:
# f. how many users have 4 phone numbers mentioned
spark.sql(''' select count(*)  from users_vw
where num_phn_numbers = 4
''')

count(1)
179041


In [38]:
#  g. how many users do not have any phone number mentioned
spark.sql(''' select count(*)  from users_vw
where user_phone_numbers is null
''')

count(1)
108981


In [23]:
# Question 3
#  Write the data from the base dataframe as it is to the disk, but write in parquet format. Observe the number of files created, also the size of files.

In [41]:
base_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv011856/w9_assignment") \
.save()

In [42]:
base_df.rdd.getNumPartitions()

5

In [28]:
# spark.stop()